# Code

Below is some python code for manipulating Turing machines. Scroll below to the palindromes section to see how it is used.

In [66]:
import copy

def initialConf (machine, word) :
    if 'onlyWorkTape' in machine:
        return {
            'state' : machine['initial'],
            'work tape' : ['⊢']+word,
            'work head' : 0
        }
    else:
        return {
            'state' : machine['initial'],
            'input tape' : ['⊢']+word+ ['⊣'],
            'input head' : 0,
            'work tape' : ['⊢'],
            'work head' : 0
        }
        

def doesTransitionApply (trans, conf) :
    
    #wrong state
    if trans['state before'] != conf['state']:
        return False

    #questions about work tape are asked only if it exists
    if 'input tape' in conf:
        #wrong input letter
        if ('input letter before' in trans) and (trans['input letter before'] != conf['input tape'][conf['input head']]):
            return False
        #input head falls out of input tape
        if ('move input head' in trans) and ((conf['input head'] + trans['move input head'] >= len(conf['input tape'])) 
                                         or (conf['input head'] + trans['move input head'] < 0)):
            return False

    #wrong work letter
    if ('work letter before' in trans) and (trans['work letter before'] != (conf['work tape'])[conf['work head']]):
        return False
    
    #work head falls out of work tape to the left
    if ('move work head' in trans) and (trans['move work head'] + conf['work head'] < 0):
        return False
    
    return True

#applies a transitions and returns the new configuration
def applyTransition(trans,conf):
    if not doesTransitionApply(trans,conf):
        raise Exception('tried to apply transition that does not apply')

    #a special configuration that halts and just gives the result, which is typically accept/reject
    if 'halt' in trans:
        return { 'halt' : trans['halt']}
    
    newconf = copy.deepcopy(conf);
    
    newconf['state'] = trans['state after']
    
    if 'move input head' in trans:
        newconf['input head']+=trans['move input head']
    
    if 'work letter after' in trans:
        newconf['work tape'][newconf['work head']] = trans['work letter after']

    if 'move work head' in trans:
        newconf['work head']+=trans['move work head']
            
    #if the work head moved out of work tape, add a new blank symbol
    if newconf['work head'] >= len(newconf['work tape']):
        newconf['work tape'] += ['']

    return newconf

#returns the list of all avaialable transitions
def availableTransitions(machine, conf):

    retval = []

    if (conf == 'accept') or (conf == 'reject'):
        return retval

    for t in machine['transitions']:
        if doesTransitionApply(t, conf):
            retval += [t]
    return retval

#returns the list of all configurations in a run, for the given input string
#if several transitions apply, the first one is used

def run(machine, string):
    
    conf = initialConf(machine, string)

    #the return value is a list of configurations
    retval = []
    while True:
        retval += [conf]

        if 'halt' in conf:
            break

        #there is a timeout of 10k transitions
        if len(retval) > 1000:
            retval+=[{'halt':'reject because run length exceeded'}]
            break
            
        transitionList = availableTransitions(machine, conf);
        
        if len(transitionList) == 0:
            retval+=[{'halt':'reject because no transition can be applied'}]
            break
        
        #we use first available transition
        t =transitionList[0]
        conf = applyTransition(t,conf)
        
        
    return retval

#this part of the code displays runs in HTML
from IPython.core.display import HTML

#returns HTML representation of a single cell
def cellHTML(content, head) :
    if head:
        return '<span style="background-color:red; padding: 5px; margin: 3px">' +  content + '</span>'
    else:
        return '<span style="background-color:lightgrey; padding: 5px; margin: 3px">' +  content + '</span>'

#returns HTML representation of a tape, which can be the input or work tape
def tapeHTML(string, head) :
    if (head < 0 or head >= len(string)):
        raise Exception('head out of bounds')
    index = 0
    retval = '<span style="margin:20px">'
    for x in string:
        retval += cellHTML(x, index == head)
        index += 1
    return retval + '</span>'

#returns HTML representation of an entire configuration
def confHTML(conf) :
    retval = '<div style="padding:20px">'
    if ('halt' in conf):
        retval += conf['halt']
    else:
        retval +='<span style="display:inline-block; width:100px"> state:' + conf['state'] + '</span>'
        #if the machine is two tape, then it has a separate input tape
        if ('input tape' in conf):
            retval += 'input tape:'
            retval += tapeHTML(conf['input tape'], conf['input head'])
            retval += 'work tape:'
        #both one and two tape machines have a work tape
        retval += tapeHTML(conf['work tape'], conf['work head'])
    
    retval += '</div>'
    return retval 

def displayConf(conf) :
    HTML(confHTML(conf))
    
def displayRun(machine,string):
    retval = ''
    for conf in run(machine, string):
        retval += confHTML(conf)
    return HTML(retval)

The syntax of Turing machines is explained below, on the example of a machine for palindromes

# Palindromes

We give two example machines for palindromes. One is the (default) two tape machine, and the other is a one tape machine.

### Two tape palindromes

Here is a (two tape) Turing machine with input alphabet ['a','b'] that accepts exactly the palindromes. 

In [15]:
machine_pals = { 'initial' : 'p',
          'transitions' : [
           {'state before' : 'p', 'state after' : 'p', 'input letter before' : '⊢','move input head' : 1,'move work head' : 1,'work letter after' : '⊢'},
              {'state before' : 'p', 'state after' : 'p', 'input letter before' : 'a', 'move input head' : 1, 'move work head' : 1,'work letter after' : 'a'},
              {'state before' : 'p', 'state after' : 'p', 'input letter before' : 'b', 'move input head' : 1, 'move work head' : 1, 'work letter after' : 'b'},
              {'state before' : 'p', 'state after' : 'q', 'input letter before' : '⊣', 'move input head' : -1},
              {'state before' : 'q', 'state after' : 'q', 'input letter before' : 'a','move input head' : -1},
              {'state before' : 'q', 'state after' : 'q', 'input letter before' : 'b', 'move input head' : -1},
              {'state before' : 'q', 'state after' : 'r', 'input letter before' : '⊢','move input head' : 1,'move work head' : -1},
              {'state before' : 'r', 'state after' : 'r', 'input letter before' : 'a','work letter before' : 'a' ,'move work head' : -1, 'move input head' : 1},
              {'state before' : 'r', 'state after' : 'r', 'input letter before' : 'b','work letter before' : 'b' ,'move work head' : -1, 'move input head' : 1},
            {'state before' : 'r', 'input letter before' : '⊣','halt' : 'accept' },
           ]
          }

A machine consists of an initial state, and a list of transitions. The states are implicit – these are the states that appear in the transitions, and also the work and input alphabet are implicit. The states are strings (and not just letters), the same is true for letters of the input and work alphabet, i.e. each such letter is a string. A transition is a record that has the following fields: 'state before', 'state after', 'input letter before', 'work letter before', 'move input head', 'move work head', 'input letter after', 'work letter after'. The fields concerning the letters (both input and work, both before and after) are optional. Omitting a '* letter before' field means the transition applies to all letters, omitting a '* letter after' field means the cell keeps its old value and is not overwritten.

In [3]:
displayRun(machine_pals,['a','b','a'])

### One tape palindromes

Here is a one tape machine that recognizes the palindromes, again over the alphabet 'a' and 'b'. In the one tape variant, there is a flag 'onlyWorkTape' : true, which is not used in the default two tape format. The work tape is the only tape.

In [4]:
oneTapeMachine = { 'onlyWorkTape' : True,
                  'initial' : 'init',
                  'transitions' : [
                      {'state before' : 'init', 'state after' : 'q', 'move work head' : 1},
                      {'state before' : 'q', 'state after' : 'senda', 'work letter before': 'a',  'work letter after': '', 'move work head' : 1,'work tape after' : ''},
                      {'state before' : 'senda', 'state after' : 'senda', 'work letter before': 'a', 'move work head' : 1},
                      {'state before' : 'senda', 'state after' : 'senda', 'work letter before': 'b', 'move work head' : 1},
                      {'state before' : 'senda', 'state after' : 'checka', 'work letter before': '', 'move work head' : -1},
                      {'state before' : 'checka', 'state after' : 'return', 'work letter before': 'a', 'work letter after': '', 'move work head' : -1, 'work tape after' : ''},
                      {'state before' : 'q', 'state after' : 'sendb', 'work letter before': 'b', 'work letter after': '', 'move work head' : 1,'work tape after' : ''},
                      {'state before' : 'sendb', 'state after' : 'sendb', 'work letter before': 'a', 'move work head' : 1},
                      {'state before' : 'sendb', 'state after' : 'sendb', 'work letter before': 'b', 'move work head' : 1},
                      {'state before' : 'sendb', 'state after' : 'checkb', 'work letter before': '', 'move work head' : -1},
                      {'state before' : 'checkb', 'state after' : 'return', 'work letter before': 'b', 'work letter after': '','move work head' : -1, 'work tape after' : ''},
                      {'state before' : 'return', 'state after' : 'return', 'work letter before': 'a', 'move work head' : -1},
                      {'state before' : 'return', 'state after' : 'return', 'work letter before': 'b', 'move work head' : -1},
                      {'state before' : 'return', 'state after' : 'q', 'work letter before': '', 'move work head' : 1},
                      {'state before' : 'q', 'work letter before': '', 'halt' : 'accept'},                  
                      {'state before' : 'checka', 'work letter before': '', 'halt' : 'accept'}, 
                      {'state before' : 'checkb', 'work letter before': '', 'halt' : 'accept'}, 
                  ]
          }

In [5]:
displayRun(oneTapeMachine,['a', 'b', 'b', 'a'])

In [67]:
def eliminateInputTape(machine):
    from string import ascii_letters, digits
    from random import choice

    class MachineInfo:
        LETTER_TRANSITIONS = ["input letter before", "work letter before", "input letter after", "work letter after"]
        CHARS = ascii_letters + digits

        def __init__(self):
            self.state_name_prefix = ""
            self.letters_set = set()
            self.states_set = set()
            self.state_copy_counter = dict()
            self.state_mapping = dict()

        def random_char(self):
            return choice(self.CHARS)

        def unused_name(self, name, name_set):
            while name in name_set:
                name = name + self.random_char()
            name_set.add(name)
            return name

        def set_state_name_prefix(self, prefix):
            self.state_name_prefix = prefix

        def valid_state_name(self, state_name):
            return self.unused_name(self.state_name_prefix + state_name + "_", self.states_set)

        def valid_letter(self, letter):
            return self.unused_name(letter, self.letters_set)

        def add_letter(self, letter):
            self.letters_set.add(letter)

        def add_state(self, state_name):
            self.states_set.add(state_name)

        def state_copy(self, state_name, copy_id):
            return self.state_mapping.get((state_name, copy_id))

        def next_state_copy(self, state_name, copy_id):
            return self.state_copy(state_name, copy_id + 1)

        def first_state_copy(self, state_name):
            return self.state_copy(state_name, 0)        

        # If many transitions from the same state occurs divide them into multiple copies.
        def add_state_copy(self, state_name):
            if state_name not in self.state_copy_counter:
                self.state_copy_counter[state_name] = -1
            self.state_copy_counter[state_name] += 1
            occurrences = self.state_copy_counter[state_name]
            new_name = self.valid_state_name(state_name + "_" + str(occurrences))
            self.state_mapping[(state_name, occurrences)] = new_name

    class Transition:
        def __init__(self, fields=None, state_before=None, state_after=None, work_letter_before=None, move_work_head=None, work_letter_after=None, halt=None):
            if fields is not None:
                self.fields = fields
                return

            self.fields = dict()
            self.fields["state before"] = state_before
            self.fields["state after"] = state_after
            self.fields["work letter before"] = work_letter_before
            self.fields["move work head"] = move_work_head
            self.fields["work letter after"] = work_letter_after
            self.fields["halt"] = halt

        def get(self, field):
            return self.fields.get(field)

        def to_dict(self):
            return {k: v for k, v in self.fields.items() if v is not None}

    machine_info = MachineInfo()
    for tr in machine["transitions"]:
        for l in MachineInfo.LETTER_TRANSITIONS:
            if l in tr:
                machine_info.add_letter(tr[l])

    # If many transitions from the same state occurs divide them into multiple copies.
    for tr in machine["transitions"]:
        if "state before" in tr:
            machine_info.add_state_copy(tr["state before"])
        if "state after" in tr:
            machine_info.add_state(tr["state after"])
    if machine["initial"] not in machine_info.state_copy_counter:
        machine_info.add_state_copy(machine["initial"])

    init_state = machine_info.valid_state_name("init")
    # Define 0/1 signs needed fo indicating a position of the work/input head.
    head_0 = machine_info.valid_letter("0")
    head_1 = machine_info.valid_letter("1")
    # Make sure that new letters and input word bound markers are present in the letters set.
    special_letters = ["⊢", "⊣", head_0, head_1]
    for sl in special_letters:
        machine_info.add_letter(sl)

    transitions = []
    # Add place for a marker after each letter to indicate input tape and work tape heads from the original machine.
    find_begin = machine_info.valid_state_name("find_begin")
    find_end = machine_info.valid_state_name("find_end")
    ready_to_copy = machine_info.valid_state_name("ready_to_copy")
    insert_empty = machine_info.valid_state_name("empty")
    check_copying_finish = machine_info.valid_state_name("check_copying_finish")
    divide_tapes_1 = machine_info.valid_state_name("divide_tapes_1")
    divide_tapes_2 = machine_info.valid_state_name("divide_tapes_2")
    divide_tapes_3 = machine_info.valid_state_name("divide_tapes_3")
    divide_tapes_4 = machine_info.valid_state_name("divide_tapes_4")
    transitions.append(Transition(state_before=init_state, state_after=find_end, move_work_head=1))
    transitions.append(Transition(state_before=find_end, state_after=check_copying_finish, work_letter_before="", move_work_head=-1))
    transitions.append(Transition(state_before=find_end, state_after=find_end, move_work_head=1))
    transitions.append(Transition(state_before=check_copying_finish, state_after=divide_tapes_1, work_letter_before=head_0, move_work_head=1))
    # Empty word case.
    transitions.append(Transition(state_before=check_copying_finish, state_after=ready_to_copy, work_letter_before="⊢"))

    letters_states = []
    for l in machine_info.letters_set:
        if l not in special_letters:
            letters_states.append((l, machine_info.valid_state_name(l + "_copy")))
            transitions.append(Transition(state_before=check_copying_finish, state_after=ready_to_copy, work_letter_before=l))
    for l, l_state in letters_states:
        transitions.append(Transition(state_before=ready_to_copy, state_after=l_state, work_letter_before=l, move_work_head=1))
        transitions.append(Transition(state_before=l_state, state_after=ready_to_copy, work_letter_after=l, move_work_head=-2))
    transitions.append(Transition(state_before=ready_to_copy, state_after=insert_empty, work_letter_before="⊢", move_work_head=1))
    transitions.append(Transition(state_before=ready_to_copy, state_after=insert_empty, work_letter_before=head_0, move_work_head=2))
    transitions.append(Transition(state_before=insert_empty, state_after=find_end, work_letter_after=head_0, move_work_head=1))

    # Finished placing original input tape, place original work tape.
    transitions.append(Transition(state_before=divide_tapes_1, state_after=divide_tapes_2, work_letter_after="⊣", move_work_head=1))
    transitions.append(Transition(state_before=divide_tapes_2, state_after=divide_tapes_3, work_letter_after=head_0, move_work_head=1))
    transitions.append(Transition(state_before=divide_tapes_3, state_after=divide_tapes_4, work_letter_after="⊢", move_work_head=1))
    transitions.append(Transition(state_before=divide_tapes_4, state_after=find_begin, work_letter_after=head_1, move_work_head=-2))

    # Move back to the beginning.
    mark_ready_to_work = machine_info.valid_state_name("mark_ready_to_work")
    ready_to_work = machine_info.valid_state_name("ready_to_work")
    transitions.append(Transition(state_before=find_begin, state_after=mark_ready_to_work, work_letter_before="⊢", move_work_head=1))
    transitions.append(Transition(state_before=find_begin, state_after=find_begin, move_work_head=-1))
    transitions.append(Transition(state_before=mark_ready_to_work, state_after=ready_to_work, work_letter_after=head_1, move_work_head=-1))

    # Add the transition to the initial state from the original machine.
    transitions.append(Transition(state_before=ready_to_work, state_after=machine_info.first_state_copy(machine["initial"])))

    state_counter = dict()
    # Assumption: before simulating each original transition current head is at the position of the original input head.
    for cnt, tr in enumerate(machine["transitions"]):
        curr_tr = Transition(tr)
        state = curr_tr.get("state before")
        if state is None:
            continue

        if state not in state_counter:
            state_counter[state] = -1
        state_counter[state] += 1
        current_count = state_counter[state]
        next_state_copy = machine_info.next_state_copy(state, current_count)
        state = machine_info.state_copy(state, current_count)

        # Used to create more user-friendly names of new states.
        machine_info.set_state_name_prefix(str(cnt) + "_")

        find_input_marker = machine_info.valid_state_name("find_input_marker")
        found_input_marker = machine_info.valid_state_name("found_input_marker")
        transitions.append(Transition(state_before=find_input_marker, state_after=found_input_marker, work_letter_before=head_1, move_work_head=-1))
        transitions.append(Transition(state_before=find_input_marker, state_after=find_input_marker, move_work_head=-1))

        find_work_marker = machine_info.valid_state_name("find_work_marker")
        found_work_marker = machine_info.valid_state_name("found_work_marker")
        transitions.append(Transition(state_before=find_work_marker, state_after=found_work_marker, work_letter_before=head_1, move_work_head=-1))
        transitions.append(Transition(state_before=find_work_marker, state_after=find_work_marker, move_work_head=1))

        orig_input_letter_before = curr_tr.get("input letter before")
        orig_input_letter_after = curr_tr.get("input letter after")
        orig_work_letter_before = curr_tr.get("work letter before")
        orig_work_letter_after = curr_tr.get("work letter after")

        orig_move_input_head = curr_tr.get("move input head")
        orig_move_work_head = curr_tr.get("move work head")
        if orig_move_input_head is None:
            orig_move_input_head = 0
        if orig_move_work_head is None:
            orig_move_work_head = 0

        reset_state = None
        if next_state_copy is not None:
            reset_state = machine_info.valid_state_name("reset_state")

        # Check if input head will not pass through the left/right side of the input tape.
        # Here we use the assumption from the task that move_input_head belongs to set {-1, 0, 1}.
        if orig_move_input_head == -1:
            if next_state_copy is not None:
                transitions.append(Transition(state_before=state, work_letter_before="⊢", state_after=next_state_copy))
            else:
                empty_state_input = machine_info.valid_state_name("empty_state_input")
                transitions.append(Transition(state_before=state, work_letter_before="⊢", state_after=empty_state_input))
        if orig_move_input_head == 1:
            if next_state_copy is not None:
                transitions.append(Transition(state_before=state, work_letter_before="⊣", state_after=next_state_copy))
            else:
                empty_state_input = machine_info.valid_state_name("empty_state_input")
                transitions.append(Transition(state_before=state, work_letter_before="⊣", state_after=empty_state_input))

        # Check if input letter matches, move head to the work tape marker.
        transitions.append(Transition(state_before=state, state_after=find_work_marker, work_letter_before=orig_input_letter_before, move_work_head=2))
        if next_state_copy is not None:
            transitions.append(Transition(state_before=state, state_after=next_state_copy))

        # Check if work head will not pass through the left side of the work tape.
        # Here we use the assumption from the task that move_work_head belongs to set {-1, 0, 1}.
        if orig_move_work_head == -1:
            if next_state_copy is not None:
                transitions.append(Transition(state_before=found_work_marker, work_letter_before="⊢", state_after=reset_state))
            else:
                empty_state_work = machine_info.valid_state_name("empty_state_work")
                transitions.append(Transition(state_before=found_work_marker, work_letter_before="⊢", state_after=empty_state_work))

        # Divide shift into three parts in order to avoid passing the right end of the work tape.
        work_head_shift_1 = machine_info.valid_state_name("work_head_shift_1")
        work_head_shift_2 = machine_info.valid_state_name("work_head_shift_2")
        work_head_shift_3 = machine_info.valid_state_name("work_head_shift_3")
        # Check work letter.
        transitions.append(Transition(state_before=found_work_marker, state_after=work_head_shift_1, work_letter_before=orig_work_letter_before, work_letter_after=orig_work_letter_after, move_work_head=1))
        # Work letter does not match: go back to the next option for the current state.
        if next_state_copy is not None:
            transitions.append(Transition(state_before=found_work_marker, state_after=reset_state))
            transitions.append(Transition(state_before=reset_state, state_after=next_state_copy, work_letter_before=head_1, move_work_head=-1))
            transitions.append(Transition(state_before=reset_state, state_after=reset_state, move_work_head=-1))

        # Move the work head marker.
        transitions.append(Transition(state_before=work_head_shift_1, state_after=work_head_shift_2, work_letter_after=head_0, move_work_head=orig_move_work_head))
        transitions.append(Transition(state_before=work_head_shift_2, state_after=work_head_shift_3, move_work_head=orig_move_work_head))
        # Go back to the input head marker.
        transitions.append(Transition(state_before=work_head_shift_3, state_after=find_input_marker, work_letter_after=head_1, move_work_head=-1))

        input_head_shift_1 = machine_info.valid_state_name("input_head_shift_1")
        input_head_shift_2 = machine_info.valid_state_name("input_head_shift_2")
        finished_transition = machine_info.valid_state_name("finished_transition")
        # Change the input letter (iff the original machine allows such changes).
        transitions.append(Transition(state_before=found_input_marker, state_after=input_head_shift_1, work_letter_after=orig_input_letter_after, move_work_head=1))
        # Move the input head marker.
        transitions.append(Transition(state_before=input_head_shift_1, state_after=input_head_shift_2, work_letter_after=head_0, move_work_head=orig_move_input_head * 2))
        # Transition finished.
        transitions.append(Transition(state_before=input_head_shift_2, state_after=finished_transition, work_letter_after=head_1, move_work_head=-1))

        orig_halt = curr_tr.get("halt")
        if orig_halt is not None:
            transitions.append(Transition(state_before=finished_transition, halt=orig_halt))
            continue

        next_state = curr_tr.get("state after")
        if next_state is not None:
            transitions.append(Transition(state_before=finished_transition, state_after=machine_info.first_state_copy(next_state)))

    return {
        "onlyWorkTape": True,
        "initial": init_state,
        "transitions": [tr.to_dict() for tr in transitions],
    }
